In [13]:
import warnings
warnings.filterwarnings('ignore')
import mpld3, seaborn as sns, matplotlib.pyplot as plt
sns.set(style='whitegrid'), sns.set_context("paper", font_scale=2., rc={"lines.linewidth": 1.6})
import pandas as pd, numpy as np, matplotlib.dates as dates, scipy.stats as sp, scipy.optimize as op, os
from datetime import datetime, date, timedelta
from matplotlib.pyplot import cm 
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
flow_folder = os.path.join(os.getcwd(), 'Kali_Gandaki_data', 'csvs')
flow_files = [x for x in os.listdir(flow_folder) if x.endswith('.csv')]
flow_dict = {}
for i in range(len(flow_files)):
    flow_dict[i] = pd.read_csv(os.path.join(flow_folder, flow_files[i]))
    rng = pd.to_datetime(flow_dict[i]['Date'])
    flow_dict[i].columns = ['date', 'flow']
    flow_dict[i] = pd.DataFrame(flow_dict[i]['flow'].tolist(), index = rng)
    flow_dict[i].columns = ['flow']

In [11]:
flow_dict = {}
for i in range(len(flow_files)):
    flow_dict[i] = pd.read_csv(os.path.join(flow_folder, flow_files[i]))
    rng = pd.to_datetime(flow_dict[i]['Date'])
    flow_dict[i].columns = ['date', 'flow']
    flow_dict[i] = pd.DataFrame(flow_dict[i]['flow'].tolist(), index = rng)
    flow_dict[i].columns = ['flow']
    
option=0;
start = 0;
selectivity = 100;
ante = 10;
window = 11;
minLen = 3;
alph = .9;

from peakdetect import peakdet as peakdet
sitesDict = {}
for key in [0]:
    d = flow_dict[key]
    selector = (d.flow.max()-d.flow.min())/selectivity;
    [maxtab, mintab]=peakdet(d.flow, selector);
    #initialize peaks
    d['peaks']=-1;

    #get rid of peaks too close to the start
    if ante>2:
        maxtab = maxtab[maxtab[:,0].astype(int)>ante];
    else:
        maxtab = maxtab[maxtab[:,0].astype(int)>1];


    d.ix[maxtab[:,0].astype(int),'peaks']=maxtab[:,1];

    d['smooth']= pd.rolling_mean(d.flow,window);
    d['smooth'][0:(window-1)] = d.flow[0:(window-1)];
    d['Dunsmooth']= d.flow.diff().shift(-1);
    d['Dsmooth']= d.smooth.diff().shift(-1);
    d['DDsmooth']=d['smooth'].diff().shift(-1).diff().shift(-1); 
    d['DDunsmooth'] = d.flow.diff().shift(-1).diff().shift(-1);
    d = d[:-(window-1)];


    #boolean vector for recession periods 
    if option==0:
        d['choose']=((d['Dsmooth']<=0)|(d['Dunsmooth']<=0));
    else:
        d['choose']=(d['Dunsmooth']<=0) & ((d['DDsmooth']>=0)|(d['DDunsmooth']>=0));
        
    d['choose'][-1] = False

    #each peak should have associated with it an API, an An, A0n, Bn, A, A0, B
    #an rsqn, and an rsq. Loop through all peaks, compute each parameter

    datesMax = d.ix[d['peaks']>0].index;
    d['rec'] = False
    d['An']=np.nan; d['Bn']=np.nan; d['A0n']=np.nan; d['api']=np.nan;
    for i in np.arange(len(datesMax)-1):
        recStart = datesMax[i]; peak1 = datesMax[i]+pd.DateOffset(days=1); peak2 = datesMax[i+1];
        
        if len(d[peak1:peak2][d[peak1:peak2]['choose']==False].index)==0:
            continue
            
        recEnd = d[peak1:peak2][d[peak1:peak2]['choose']==False].index[0];
        if len(d[recStart:recEnd])<minLen:
            continue;
        t = np.arange(len(d.flow[recStart:recEnd]));
        q0 = d.flow[recStart];
        def func(t, a, r):
            return (q0**(r)-a*r*t)**(1/r)
        try:
            popt, cov = op.curve_fit(func,t,d.flow[recStart:recEnd]);
        except RuntimeError:
            continue;
        afit = popt[0]; bfit = 1-popt[1];
        if bfit>0:
            d['An'][recStart] = afit; d['Bn'][recStart]=bfit;
            beforeRec = d.flow[recStart-pd.DateOffset(days=ante):recStart];
            beforeRec = beforeRec[::-1]
            factor = alph**np.arange(len(beforeRec))
            api = np.sum(beforeRec*factor)
            d['api'][recStart] = api
            d['rec'][recStart:recEnd]=True


    notAn = d['An'][~np.isnan(d['An'])];
    notBn = d['Bn'][~np.isnan(d['Bn'])];

    ag = sp.gmean(notAn)
    exponent = np.sum((notBn-np.mean(notBn))*np.log10(notAn/ag))/sum((notBn-np.mean(notBn))**2)
    qscale = 10**(-exponent)
    d['A0n'][~np.isnan(d['An'])]=d['An'][~np.isnan(d['An'])]*qscale**(d['Bn'][~np.isnan(d['An'])]-1)
    sitesDict[key]=d;

In [14]:
fig = plt.figure()
plt.plot(d.index, d['flow']); plt.scatter(d.index[d.rec], d.loc[d.rec, 'flow'], color='r')
plt.plot(d.index, d['smooth'],color='g',ls='--')
mpld3.display(fig)